In [1]:
!pip install -U datasets

In [2]:
!rm -rf ~/.cache/huggingface/datasets/pythainlp___thaisum

In [3]:
import torch

In [4]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็นทั้งหมด
!pip install -q transformers[torch] datasets accelerate pythainlp sentencepiece yake

import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.optim import AdamW # Corrected import for AdamW
from datasets import load_dataset
from tqdm.auto import tqdm
import yake
from pythainlp.tokenize import word_tokenize

# ตรวจสอบและตั้งค่าอุปกรณ์ (GPU)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")

อุปกรณ์ที่ใช้: cuda


In [17]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด ThaiSum Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face (เลือกเฉพาะส่วนของข่าว)
dataset = load_dataset("pythainlp/thaisum", "default", split='train', streaming=True)

# Collect data from the streaming dataset
data_list = []
for example in dataset:
    # Filter out examples where 'body' or 'summary' is None
    if example['body'] is not None and example['summary'] is not None:
        data_list.append({
            'body': example['body'],
            'summary': example['summary'],
            'tags': example['tags']
        })
    # Optional: Limit the number of examples for faster testing
    if len(data_list) >= 6000: # Collect slightly more than needed for sampling
        break

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)


# --- สร้างข้อมูลสำหรับฝึกซ้อม (train.csv) ---
# เลือกเฉพาะคอลัมน์ที่ต้องการและเปลี่ยนชื่อให้ตรงกับโจทย์
train_df = df[['body', 'summary', 'tags']].copy()
train_df.rename(columns={
    'body': 'document_text',
    'summary': 'summary',
    'tags': 'keywords'
}, inplace=True)

# ทำความสะอาด keywords (แปลง string ของ list ให้เป็น string ที่คั่นด้วย comma)
train_df['keywords'] = train_df['keywords'].apply(lambda x: ', '.join(eval(x)) if isinstance(x, str) and x.startswith('[') else '')
train_df = train_df.dropna().sample(n=5000, random_state=42).reset_index(drop=True) # Added .reset_index(drop=True)


# --- สร้างข้อมูลสำหรับทดสอบ (test.csv) ---
# Create test_df from the collected data before sampling for train_df
test_df_data = [d for i, d in enumerate(data_list) if i not in train_df.index] # Exclude indices used for train_df
test_df = pd.DataFrame(test_df_data)
test_df = test_df[['body']].copy() # Select only the 'body' column
test_df.rename(columns={'body': 'document_text'}, inplace=True)
test_df = test_df.sample(n=500, random_state=42).reset_index(drop=True) # Sample 500 examples
test_df['document_id'] = test_df.index


print(f"เตรียมข้อมูล train_df สำเร็จ จำนวน {len(train_df)} ตัวอย่าง")
print("ตัวอย่างข้อมูล Train:")
print(train_df.head())

print(f"\nเตรียมข้อมูล test_df สำเร็จ จำนวน {len(test_df)} ตัวอย่าง")
print("ตัวอย่างข้อมูล Test:")
test_df.head()

กำลังโหลด ThaiSum Dataset...
เตรียมข้อมูล train_df สำเร็จ จำนวน 5000 ตัวอย่าง
ตัวอย่างข้อมูล Train:
                                       document_text  \
0  สำนักข่าวต่างประเทศรายงานวันที่ 8 พ.ย. ว่า ซลา...   
1  วันนี้ (3 มิ.ย.2563) พ.ต.อ.กฤษณะ พัฒนเจริญ รอง...   
2  ผู้สื่อข่าวรายงานว่า เมื่อวันที่ 20 เม.ย.ที่ผ่...   
3  เมื่อวันที่ 6 มิ.ย. 60 ที่เรือนจำกลางขอนแก่น น...   
4  คาดว่าจะเคลื่อนเข้ามาปกคลุมบริเวณเกาะไหหลำ ประ...   

                                             summary keywords  
0  ซลาตัน อิบราฮิโมวิช ยอดดาวยิงของ ผีแดง แมนเชสเ...           
1  ศาลค้านประกันตัวพ่อเลี้ยง ที่ถูกกล่าวหาข่มขืนน...           
2  สำนักงานคณะกรรมการกำกับหลักทรัพย์และตลาดหลักทร...           
3  ผบ.เรือนจำกลางขอนแก่น เผย เปรี้ยว นอนคุกคืนแรก...           
4  เมื่อวันที่ 13 ก.ย.2561 กรมอุตุนิยมวิทยา รายงา...           

เตรียมข้อมูล test_df สำเร็จ จำนวน 500 ตัวอย่าง
ตัวอย่างข้อมูล Test:


,document_text,document_id
0,ความจริงมีเพียงสองประเทศที่ได้รับการแจ้งถึงเห...,0
1,เมื่อวันที่ 28 พ.ค.62 ที่บก.ปคม. พล.ต.ต.วรวัฒน...,1
2,วันที่ 14 พ.ย. ผู้สื่อข่าวรายงานว่า ที่ถนนสายต...,2
3,ชุดปส.ตำรวจหาดใหญ่ บุกจับเอเย่นต์ยาเสพติดรายให...,3
4,ที่ผ่านมา โดยระบุว่าข้อสอบยากเกินความสามารถของ...,4


In [14]:
# %% 3. การโหลด Tokenizer และโมเดล mT5

MODEL_NAME = 'google/mt5-small'

print(f"กำลังโหลด Tokenizer และ Model จาก '{MODEL_NAME}'...")
# โหลด Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# โหลด Model สำหรับงาน Seq2Seq
model_summarize = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

print("โหลด Tokenizer และ Model สำเร็จ")


กำลังโหลด Tokenizer และ Model จาก 'google/mt5-small'...


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

โหลด Tokenizer และ Model สำเร็จ


In [18]:
# %% 4. การประมวลผลข้อมูลสำหรับโมเดล

class SummarizationDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_max_len=512, target_max_len=128):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_max_len = source_max_len
        self.target_max_len = target_max_len
        self.source_text = self.data['document_text']
        self.target_text = self.data['summary']

    def __len__(self):
        return len(self.target_text)

    def __getitem__(self, index):
        source_text = "summarize: " + str(self.source_text[index])
        target_text = str(self.target_text[index])

        # Tokenize ข้อความต้นฉบับ
        source = self.tokenizer.batch_encode_plus(
            [source_text], max_length=self.source_max_len, padding='max_length',
            truncation=True, return_tensors='pt'
        )
        # Tokenize บทสรุป (เป็น Label)
        target = self.tokenizer.batch_encode_plus(
            [target_text], max_length=self.target_max_len, padding='max_length',
            truncation=True, return_tensors='pt'
        )

        return {
            'input_ids': source['input_ids'].flatten(),
            'attention_mask': source['attention_mask'].flatten(),
            'labels': target['input_ids'].flatten()
        }

# สร้าง Dataset และ DataLoader
train_dataset = SummarizationDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

print("สร้าง Dataset และ DataLoader สำหรับการฝึกสอนสำเร็จ")


สร้าง Dataset และ DataLoader สำหรับการฝึกสอนสำเร็จ


In [19]:
# %% 5. การ Fine-tune โมเดลสรุปความ

# ตั้งค่า Optimizer
optimizer = AdamW(model_summarize.parameters(), lr=1e-4)

# กำหนดให้โมเดลอยู่ในโหมดฝึกสอน
model_summarize.train()

print("เริ่มต้นการ Fine-tune โมเดลสรุปความ...")
# สำหรับการทดลอง จะฝึกแค่ 1 Epoch (ในการแข่งจริง อาจต้องใช้ 3-5 Epochs)
for epoch in range(1):
    print(f"--- Epoch {epoch + 1} ---")
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
        # นำข้อมูลเข้า GPU
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        # คำนวณ Loss
        outputs = model_summarize(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

print(f"\nFine-tuning เสร็จสิ้น! Loss สุดท้าย: {loss.item()}")

# บันทึกโมเดลที่ฝึกเสร็จแล้ว
model_summarize.save_pretrained("./mt5_summarizer_finetuned")
tokenizer.save_pretrained("./mt5_summarizer_finetuned")
print("โมเดลที่ Fine-tune แล้วถูกบันทึกที่ './mt5_summarizer_finetuned'")


เริ่มต้นการ Fine-tune โมเดลสรุปความ...
--- Epoch 1 ---


Training Epoch 1:   0%|          | 0/1250 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.



Fine-tuning เสร็จสิ้น! Loss สุดท้าย: 2.8293588161468506
โมเดลที่ Fine-tune แล้วถูกบันทึกที่ './mt5_summarizer_finetuned'


In [20]:
# %% 6. การสกัดคีย์เวิร์ดด้วย YAKE

# ตั้งค่า YAKE สำหรับภาษาไทย
language = "th"
max_ngram_size = 1 # กำหนดให้คีย์เวิร์ดเป็นคำเดียว
deduplication_threshold = 0.9
num_of_keywords = 10 # จำนวนคีย์เวิร์ดที่ต้องการ

# สร้าง custom extractor โดยใช้การตัดคำจาก pythainlp
custom_kw_extractor = yake.KeywordExtractor(
    lan=language, n=max_ngram_size, dedupLim=deduplication_threshold,
    top=num_of_keywords, features=None
)

def extract_keywords_yake(text):
    """
    ฟังก์ชันสำหรับสกัดคีย์เวิร์ดโดยใช้ YAKE และ pythainlp
    """
    # ตัดคำด้วย pythainlp ก่อน
    words = word_tokenize(text, engine='newmm')
    # นำคำที่ตัดแล้วมาต่อกันด้วย space
    tokenized_text = " ".join(words)
    # สกัดคีย์เวิร์ดจากข้อความที่ตัดคำแล้ว
    keywords = custom_kw_extractor.extract_keywords(tokenized_text)
    # คืนค่าเฉพาะตัวคีย์เวิร์ด
    return [kw for kw, score in keywords]

# --- ทดลองใช้ฟังก์ชัน ---
sample_text = test_df['document_text'].iloc[0]
extracted_kws = extract_keywords_yake(sample_text)

print("ทดลองสกัดคีย์เวิร์ดจากเอกสารตัวอย่าง:")
print("\nข้อความตัวอย่าง:", sample_text[:300] + "...")
print("\nคีย์เวิร์ดที่สกัดได้:", extracted_kws)


ทดลองสกัดคีย์เวิร์ดจากเอกสารตัวอย่าง:

ข้อความตัวอย่าง:  ความจริงมีเพียงสองประเทศที่ได้รับการแจ้งถึงเหตุภัยพิบัติ ในขณะที่ประเทศอื่นๆ ซึ่งมีโอกาสประสบภัยนี้ไม่ต่างกัน กลับไม่ได้รับการแจ้งเตือน จึงเป็นข้อกังขาใหญ่จากประเทศอื่นๆ ที่ตกอยู่ในหายนะภัยเช่นกันว่า NOAAได้ใช้ความพยายามอย่างถึงที่สุดในการดำเนินการต่อการแจ้งเตือนภัยหรือไม่ นางสโนวี่กล่าวนางสโนวี่ยั...

คีย์เวิร์ดที่สกัดได้: ['การ', 'ที่', 'ไม่', 'ได้', 'ของ', 'แผ่นดินไหว', 'แจ้ง', 'ซึ่ง', 'ประเทศ', 'จาก']


### Submission

In [21]:
# %% 7. การสร้างผลลัพธ์สำหรับ Test Set

print("กำลังโหลดโมเดลสรุปความที่ Fine-tune แล้ว...")
# โหลดโมเดลและ tokenizer ที่บันทึกไว้
tokenizer = AutoTokenizer.from_pretrained("./mt5_summarizer_finetuned")
model_summarize = AutoModelForSeq2SeqLM.from_pretrained("./mt5_summarizer_finetuned").to(DEVICE)
model_summarize.eval() # กำหนดให้โมเดลอยู่ในโหมดประเมินผล

results = []

print("กำลังสร้าง Summary และ Keywords สำหรับ Test Set...")
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    document_id = row['document_id']
    document_text = row['document_text']

    # --- 1. สร้าง Summary ---
    input_text = "summarize: " + document_text
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True).to(DEVICE)
    with torch.no_grad():
        summary_ids = model_summarize.generate(
            inputs, max_length=150, num_beams=4, early_stopping=True
        )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # --- 2. สกัด Keywords ---
    keywords_list = extract_keywords_yake(document_text)
    keywords_str = ", ".join(keywords_list) # แปลง list เป็น string

    # เก็บผลลัพธ์
    results.append({
        'document_id': document_id,
        'summary': summary,
        'keywords': keywords_str
    })

print("สร้างผลลัพธ์สำหรับ Test Set เสร็จสิ้น")


กำลังโหลดโมเดลสรุปความที่ Fine-tune แล้ว...
กำลังสร้าง Summary และ Keywords สำหรับ Test Set...


  0%|          | 0/500 [00:00<?, ?it/s]

สร้างผลลัพธ์สำหรับ Test Set เสร็จสิ้น


In [22]:
# %% 8. การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

# แปลง list ของผลลัพธ์เป็น DataFrame
submission_df = pd.DataFrame(results)

print("\nตัวอย่างผลลัพธ์ในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission.csv", index=False)

print("\nสร้างไฟล์ submission.csv สำเร็จ!")


กำลังสร้างไฟล์ submission.csv...

ตัวอย่างผลลัพธ์ในไฟล์ Submission:
   document_id                                            summary  \
0            0                           <extra_id_0> ท.ธ..ก. . .   
1            1                 <extra_id_0> ตร. ตร. ตร. ตร. .ก. .   
2            2                                <extra_id_0>          
3            3  <extra_id_0> ตร.ม. . ตํารวจ . ตํารวจ ตํารวจ . ...   
4            4  <extra_id_0> สพท..สทศ.. ท.. ท.ศ..ส.ท. . ท. . ท...   

                                            keywords  
0  การ, ที่, ไม่, ได้, ของ, แผ่นดินไหว, แจ้ง, ซึ่...  
1  ได้, นาย, โชติ, วันที่, เมื่อ, ที่, อายุ, จับก...  
2  ที่, ว่า, ได้, เป็น, อุบัติเหตุ, ตาก, ซึ่ง, จุ...  
3  ได้, และ, กิตติ, ให้, สามี, ที่, อยู่, ไม่, ยา...  
4  ข้อสอบ, ที่, ว่า, หลักสูตร, การ, ซึ่ง, ยาก, เก...  

สร้างไฟล์ submission.csv สำเร็จ!


In [23]:
submission_df

,document_id,summary,keywords
0,0,<extra_id_0> ท.ธ..ก. . .,"การ, ที่, ไม่, ได้, ของ, แผ่นดินไหว, แจ้ง, ซึ่..."
1,1,<extra_id_0> ตร. ตร. ตร. ตร. .ก. .,"ได้, นาย, โชติ, วันที่, เมื่อ, ที่, อายุ, จับก..."
2,2,<extra_id_0>,"ที่, ว่า, ได้, เป็น, อุบัติเหตุ, ตาก, ซึ่ง, จุ..."
3,3,<extra_id_0> ตร.ม. . ตํารวจ . ตํารวจ ตํารวจ . ...,"ได้, และ, กิตติ, ให้, สามี, ที่, อยู่, ไม่, ยา..."
4,4,<extra_id_0> สพท..สทศ.. ท.. ท.ศ..ส.ท. . ท. . ท...,"ข้อสอบ, ที่, ว่า, หลักสูตร, การ, ซึ่ง, ยาก, เก..."
...,...,...,...
495,495,<extra_id_0>จ.จ.จ.จ.จ.จ.,"และ, นครสวรรค์, หมายเลข, อ่างทอง, สะพาน, บริเว..."
496,496,<extra_id_0> สายการบินไทย ไลอ้อน แอร์,"BOEING, MAX, แม็กซ์, โบอิ้ง, เครื่องบิน, การ, ..."
497,497,<extra_id_0> ท.. ขบวนการชุมนุม,"การ, ชุมนุม, ผู้ประท้วง, ฮ่องกง, ให้, และ, Wan..."
498,498,<extra_id_0> ท.ต. . . นายก . . - . . - . . . . .,"และ, ตรัง, เทศบาลนคร, วันที่, ให้, ได้, ที่, ส..."
